In [ ]:
!pip install openai==0.28 python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [ ]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()
# Set up your OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def classify_text(input_text):
    prompt = f"""
    This is an overall sentiment classifier for movie reviews.
First, present CLUES (i.e., keywords, phrases, contextual information, semantic relations, semantic meaning,
tones, references) that support the sentiment determination of input.
Second, deduce a diagnostic REASONING process from premises (i.e., clues, input) that supports the sentiment
determination (Limit the number of words to 130).
Third, determine the overall SENTIMENT of INPUT as Positive or Negative considering CLUES, the REASONING
process and the INPUT.
INPUT: {input_text}
    """

    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=150,
        temperature=0.7
    )

    try:
        response = response.choices[0].text
        sentiment = response.lower().split("SENTIMENT:")[-1].strip()
        if "neg" in sentiment:
            sentiment = 0
        elif "pos" in sentiment:
            sentiment = 1
        else:
            print(response)
            sentiment = None
    except:
        print(response)
        sentiment = None

    return sentiment

# Example usage
text_input = "Delete the key"
print(classify_text(text_input))

0


In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:

from datasets import load_dataset

sst = load_dataset("SetFit/sst2")['test']
sst

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/378 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

dev.jsonl:   0%|          | 0.00/136k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/281k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6920 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 1821
})

In [ ]:
carp_classification = {"text":[], "new_label":[], "orig_label":[]}

for i, (text, orig_label) in enumerate(zip(sst['text'], sst['label'])):
    new_label = classify_text(text)
    carp_classification['text'].append(text)
    carp_classification['new_label'].append(new_label)
    carp_classification['orig_label'].append(orig_label)
    print(i, text, new_label, orig_label)

0 no movement , no yuks , not much of anything . 0 0
1 a gob of drivel so sickly sweet , even the eager consumers of moore 's pasteurized ditties will retch it up like rancid crème brûlée . 0 0
2 gangs of new york is an unapologetic mess , whose only saving grace is that it ends by blowing just about everything up . 0 0
3 we never really feel involved with the story , as all of its ideas remain just that : abstract ideas . 0 0
4 this is one of polanski 's best films . 1 1
5 take care of my cat offers a refreshingly different slice of asian cinema . 1 1
6 acting , particularly by tambor , almost makes `` never again '' worthwhile , but -lrb- writer\/director -rrb- schaeffer should follow his titular advice 1 0
7 the movie exists for its soccer action and its fine acting . 1 1
8 arnold 's jump from little screen to big will leave frowns on more than a few faces . 0 0
9 if this holiday movie is supposed to be a gift , somebody unwrapped it early , took out all the good stuff , and left be

In [ ]:
import pickle

with open("carp_classification.pkl", 'wb') as file:
    pickle.dump(carp_classification, file)

In [ ]:
import pandas as pd

# Creating the DataFrame with specified column names
df = pd.DataFrame(carp_classification, columns=['text', 'new_label', 'orig_label'])
df.head()

,text,new_label,orig_label
0,"no movement , no yuks , not much of anything .",0.0,0
1,"a gob of drivel so sickly sweet , even the eag...",0.0,0
2,"gangs of new york is an unapologetic mess , wh...",0.0,0
3,"we never really feel involved with the story ,...",0.0,0
4,this is one of polanski 's best films .,1.0,1


In [ ]:
df.to_csv("carp_classification.csv", index=False)

In [ ]:
!pip install numpy


In [ ]:
import numpy as np

In [ ]:
accuracy = np.sum([1 if l1==l2 else 0 for l1,l2 in zip(df['new_label'],df['orig_label']) ])
accuracy/len(df)

0.727622185612301